In [1]:
#Import dependencies
import cv2
from PIL import Image
import os
import random
import numpy as np
from matplotlib import pyplot as plt

#Import tensorflow dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
def make_embedding():
    input = Input(shape=(224,224,3), name='flood_image')

    #First convulutional layer
    c1 = Conv2D(64, (10,10), activation='relu')(input)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    #Second convulutional layer
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

    #Third and final convulutional layer
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

    #Final layer
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)

    return Model(inputs=[input], outputs=[d1], name='embedding')

In [3]:
embedding = make_embedding()
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flood_image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 215, 215, 64)      19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 108, 108, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 102, 102, 128)     401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 51, 51, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 128)       26

In [4]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [5]:
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(224,224,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(224,224,3))
    
    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [17]:
# Setup paths
CORE_PATH = '/content/drive/MyDrive/Third_Year_Project/Siamese_Network/Data_Pre_Processing_Siamese/Data_Final'
TRUE_PAIRINGS = '/content/drive/MyDrive/Third_Year_Project/Siamese_Network/Data_Pre_Processing_Siamese/Data_Final/Image_Pairs_Equal'
FALSE_PAIRINGS = '/content/drive/MyDrive/Third_Year_Project/Siamese_Network/Data_Pre_Processing_Siamese/Data_Final/Image_Pairs'

def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 224x224x3
    img = tf.image.resize(img, (224,224))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

def preprocess_twin(input_img, validation_img):
    return(preprocess(input_img), preprocess(validation_img))

In [26]:
img_height = 224
img_width = 224
channels = 3
batch_size = 10

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    CORE_PATH,
    labels="inferred",
    label_mode="int",  # categorical, binary
    # class_names=['0', '1', '2', '3', ...]
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_height, img_width),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="training",
)

print('-----------------------------------------------------------')

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    CORE_PATH,
    labels="inferred",
    label_mode="int",  # categorical, binary
    # class_names=['0', '1', '2', '3', ...]
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_height, img_width),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="training",
)

print('-----------------------------------------------------------')

print(ds_train)

#samples = ds_train.as_numpy_iterator()

#print(samples.next())

#ds_train = ds_train.map(preprocess_twin)

# Custom Loops
for epochs in range(10):
    for x, y in ds_train:   
        print(x)
        print('-------------------------------')
        print(y)
        break

    break

Found 1074 files belonging to 2 classes.
Using 967 files for training.
-----------------------------------------------------------
Found 1074 files belonging to 2 classes.
Using 967 files for training.
-----------------------------------------------------------
<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>
tf.Tensor(
[[[[168.04593   168.15306   174.67857  ]
   [202.56633   193.4847    191.08163  ]
   [172.83163   188.13266   195.20918  ]
   ...
   [ 87.52057    59.93373    62.561325 ]
   [120.15297    69.41835    76.77576  ]
   [172.42944   136.61241   120.54762  ]]

  [[202.46939   195.09184   190.43878  ]
   [212.65817   205.04083   197.96939  ]
   [178.5357    183.87753   188.18367  ]
   ...
   [155.38362    85.25542   105.42412  ]
   [ 98.11213    82.50506    78.97956  ]
   [141.32788   114.04697   105.17947  ]]

  [[192.82654   189.04593   183.31122  ]
   [217.5       214.44897   210.38776  ]
   [120.13776   126.73469   153.47958  ]
   ...
  